In [1]:
import warnings
import os
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


2025-01-08 16:27:59.094599: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736346479.115203   27566 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736346479.121761   27566 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 660894427606256164
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7229997056
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3509212640171858715
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:27:00.0, compute capability: 6.1"
xla_global_id: 416903419
]
Num GPUs Available:  1


I0000 00:00:1736346483.132174   27566 gpu_device.cc:2022] Created device /device:GPU:0 with 6895 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:27:00.0, compute capability: 6.1


In [3]:
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from skimage.color import rgb2gray, rgba2rgb
from matplotlib.image import imread
#!pip install -q keras
from keras.models import Sequential
# from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation, ReLU, Dropout
from tensorflow.keras.utils import to_categorical

class_names = ['flute', 'viola', 'cello', 'oboe', 'trumpet', 'saxophone']
number_of_classes = len(class_names)

instruments = {'flute':1, 'viola':2, 'cello':3, 'oboe':4, 'trumpet':5, 'saxophone':6}

files = glob.glob('BasicsMusicalInstrumClassifi/audio/london_phill_dataset_multi/spectrograms2/*.png')
np.random.shuffle(files)
data, labels = [], []

loaded = 0

print('Number of samples {0}'.format(len(files)))

for file in files:
  img, name = imread(file), os.path.basename(file).split('_')[0]#file.split('/')[-1].split('_')[0]
  img=rgba2rgb(img)
  data.append(rgb2gray(img))
  labels.append(instruments[name] - 1)

  loaded += 1
  if loaded % 200 == 0:
    print(loaded)

  if loaded == 5000:
    break

data, labels = np.array(data), np.array(labels)

s_60p, s_20p = int(len(data) * 0.6), int(len(data) * 0.2)

X_train, y_train = data[:s_60p], labels[:s_60p] #60%
X_validation, y_validation = data[s_60p: s_60p + s_20p], labels[s_60p: s_60p + s_20p]  #20%
X_test, y_test = data[s_60p + s_20p:], labels[s_60p + s_20p:] #20%

print(X_train.shape, X_test.shape)


Number of samples 600
200
400
600
(360, 540, 1080) (120, 540, 1080)


In [5]:
print(X_validation.shape, y_validation.shape)

(120, 540, 1080) (120,)


In [6]:
print (X_train.shape[0])

360


In [7]:
X_train = X_train.reshape(X_train.shape[0], 540, 1080,1)
X_test = X_test.reshape(X_test.shape[0], 540, 1080, 1)
X_validation = X_validation.reshape(X_validation.shape[0], 540, 1080, 1)

print('Dataset has been loaded succesfully.')
print(X_train.shape, y_train.shape)
print(X_validation.shape, y_validation.shape)

print(to_categorical(y_train).shape)
print(to_categorical(y_validation).shape)
print(X_train.shape[1:])

epochs = 30

model = Sequential([
        Conv2D(32, (3, 3), input_shape=(540, 1080, 1)),
        Activation('relu'),
        Conv2D(32, (3, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(3, 3)),
        Dropout(0.25),

        Conv2D(64, (3,3), padding='same'),
        Activation('relu'),
        Conv2D(64, (3, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(3, 3)),
        Dropout(0.25),

        Conv2D(128, (3, 3), padding='same'),
        Activation('relu'),
        Conv2D(128, (3, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(3, 3)),
        Dropout(0.25),

        Flatten(),
        Dense(128),
        Activation('relu'),
        Dropout(0.5),
        Dense(number_of_classes),
        Activation('softmax')
    ])


Dataset has been loaded succesfully.
(360, 540, 1080, 1) (360,)
(120, 540, 1080, 1) (120,)
(360, 6)
(120, 6)
(540, 1080, 1)


I0000 00:00:1736346507.047252   27566 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6895 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:27:00.0, compute capability: 6.1


In [9]:
model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 538, 1078, 32)  │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 538, 1078, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 536, 1076, 32)  │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 536, 1076, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 178, 358, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 178, 358, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 178, 358, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 178, 358, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 176, 356, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 176, 356, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 58, 118, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 58, 118, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 58, 118, 128)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 58, 118, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 116, 128)   │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 56, 116, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 18, 38, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 18, 38, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 87552)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,206,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 6)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,493,990 (43.85 MB)

 Trainable params: 11,493,990 (43.85 MB)

 Non-trainable params: 0 (0.00 B)

None


In [10]:
print('Fitting started...')
history = model.fit(X_train, to_categorical(y_train), epochs=20, validation_data=(X_validation, to_categorical(y_validation)), batch_size=4)

Fitting started...


2025-01-08 16:28:36.380765: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 839808000 exceeds 10% of free system memory.
2025-01-08 16:28:37.006470: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 839808000 exceeds 10% of free system memory.


Epoch 1/20


I0000 00:00:1736346519.280752   27678 service.cc:148] XLA service 0x7b7b5400fdc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1736346519.281125   27678 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1080, Compute Capability 6.1
I0000 00:00:1736346519.696224   27678 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-01-08 16:28:44.587919: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:306] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.02GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-01-08 16:28:47.509577: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[32,32,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[4,32,538,1078]{3,2,1,0}, f32[4,32,536,1076]{3,2,1,0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn

 1/90 ━━━━━━━━━━━━━━━━━━━━ 22:38 15s/step - accuracy: 0.0000e+00 - loss: 1.8440

I0000 00:00:1736346532.661558   27678 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


90/90 ━━━━━━━━━━━━━━━━━━━━ 29s 149ms/step - accuracy: 0.1370 - loss: 1.8683 - val_accuracy: 0.1583 - val_loss: 1.7920
Epoch 2/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 9s 104ms/step - accuracy: 0.1659 - loss: 1.7915 - val_accuracy: 0.1583 - val_loss: 1.7928
Epoch 3/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 9s 105ms/step - accuracy: 0.1450 - loss: 1.7919 - val_accuracy: 0.1583 - val_loss: 1.7932
Epoch 4/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 10s 106ms/step - accuracy: 0.1867 - loss: 1.7906 - val_accuracy: 0.1583 - val_loss: 1.7936
Epoch 5/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 10s 107ms/step - accuracy: 0.1876 - loss: 1.7902 - val_accuracy: 0.1583 - val_loss: 1.7944
Epoch 6/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 10s 105ms/step - accuracy: 0.2170 - loss: 1.7886 - val_accuracy: 0.1583 - val_loss: 1.7946
Epoch 7/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 9s 104ms/step - accuracy: 0.2111 - loss: 1.7895 - val_accuracy: 0.1583 - val_loss: 1.7950
Epoch 8/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 9s 105ms/step - accuracy: 0.1629 - loss: 1.7908 - val_accuracy: 0.1583 - 

In [ ]:
#model accuracy
print(history.history)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

#model loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

In [ ]:
scores = model.evaluate(X_test, to_categorical(y_test), verbose=1)
print(scores)